In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import scipy as sp
import matplotlib as mpl
import seaborn as sns

# Setting up Pandas
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

# Setting up Seaborn
sns.set_style("whitegrid")
#sns.set_context("poster")

%matplotlib inline

In [3]:
#Reading the data

In [2]:
train = pd.read_csv('training_data.csv')
test = pd.read_csv('test_data.csv')

In [4]:
#Dropping unneccessary columns

In [6]:
training = train.drop('status_group', axis=1)

In [7]:
training = training.drop('Unnamed: 0', axis=1)
test = test.drop('Unnamed: 0', axis=1)

In [8]:
training.shape

(59400, 21)

In [9]:
training.head()

,amount_tsh,days_since_recorded,funder,gps_height,installer,basin,subvillage,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type,payment_type,water_quality,quantity_group,source_type,source_class,waterpoint_type,waterpoint_type_group
0,6000.0,995,other,1390,other,Lake Nyasa,Mnyusi B,109,True,vwc,Roman,False,90s,gravity,annually,soft,enough,spring,groundwater,communal standpipe,communal standpipe
1,0.0,272,other,1399,other,Lake Victoria,Nyamara,280,Unknown,other,Others,True,10s,gravity,never pay,soft,insufficient,rainwater harvesting,surface,communal standpipe,communal standpipe
2,25.0,281,other,686,other,Pangani,Majengo,250,True,vwc,Nyumba ya mungu pipe scheme,True,00s,gravity,per bucket,soft,enough,dam,surface,communal standpipe multiple,communal standpipe
3,0.0,309,Unicef,263,other,Ruvuma / Southern Coast,Mahakamani,58,True,vwc,Others,True,80s,submersible,never pay,soft,dry,borehole,groundwater,communal standpipe multiple,communal standpipe
4,0.0,874,other,0,other,Lake Victoria,Kyanyamisa,0,True,other,Others,True,unknown,gravity,never pay,soft,seasonal,rainwater harvesting,surface,communal standpipe,communal standpipe


In [7]:
test.shape

(14850, 21)

In [11]:
##Plot function for Confusion Matrix

#plt.rcParams['figure.figsize'] = (6.0, 6.0)
from sklearn.metrics import confusion_matrix
labels=['functional','functional needs repair','non functional']
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar(shrink=0.7)
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45, ha='right', fontsize=12)
    plt.yticks(tick_marks, labels , fontsize=12)
    plt.tight_layout()
    plt.ylabel('True label', fontsize=12)
    plt.xlabel('Predicted label', fontsize=12)

# Because we are having maximum amount of Categorical data let us transform the features, and convert them into list

In [14]:
def transform_feature(df, column_name):
    unique_values = set(df[column_name].tolist())
    transformer_dict = {}
    for index, value in enumerate(unique_values):
        transformer_dict[value] = index
    df[column_name] = df[column_name].apply(lambda y: transformer_dict[y])
    return df

In [17]:
ll=set(training['funder'].tolist())
print(ll)

{'rwssp', 'other', 'hesawa', 'world_bank', 'Kkkt', 'danida', 'District Council', 'Tasaf', 'gov', 'Unicef', 'World Vision'}


In [18]:
transform_dict={}

In [19]:
for index,value in enumerate(ll):
    print(index,value)
                        

0 rwssp
1 other
2 hesawa
3 world_bank
4 Kkkt
5 danida
6 District Council
7 Tasaf
8 gov
9 Unicef
10 World Vision


In [13]:
training.columns

Index(['amount_tsh', 'days_since_recorded', 'funder', 'gps_height', 'installer', 'basin', 'subvillage', 'population', 'public_meeting', 'scheme_management', 'scheme_name', 'permit', 'construction_year', 'extraction_type', 'payment_type', 'water_quality', 'quantity_group', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group'], dtype='object')

In [15]:
integer_columns = ['days_since_recorded', 'population','gps_height'] 
columns_to_transform = [col for col in training.columns if col not in integer_columns]
for column in columns_to_transform: 
    training = transform_feature(training, column)
    test = transform_feature(test, column)

In [21]:
training.head()

,amount_tsh,days_since_recorded,funder,gps_height,installer,basin,subvillage,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type,payment_type,water_quality,quantity_group,source_type,source_class,waterpoint_type,waterpoint_type_group
0,77,995,1,1390,1,2,18779,109,1,7,1182,2,6,5,4,3,2,2,2,6,5
1,0,272,1,1399,1,8,12064,280,0,0,248,1,5,5,0,3,4,1,1,6,5
2,15,281,1,686,1,0,9296,250,1,7,926,1,2,5,6,3,2,6,1,3,5
3,0,309,9,263,1,3,11452,58,1,7,248,1,3,7,0,3,1,5,2,3,5
4,0,874,1,0,1,8,4317,0,1,0,248,1,0,5,0,3,3,1,1,6,5


# LabelEncoder also does the same thing

In [16]:
select=[cols for cols in training.columns if cols not in integer_columns]
select

['amount_tsh',
 'funder',
 'installer',
 'basin',
 'subvillage',
 'public_meeting',
 'scheme_management',
 'scheme_name',
 'permit',
 'construction_year',
 'extraction_type',
 'payment_type',
 'water_quality',
 'quantity_group',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group']

In [17]:
from sklearn.preprocessing import LabelEncoder
training[select]=training[select].apply(LabelEncoder().fit_transform)
test[select]=test[select].apply(LabelEncoder().fit_transform)

In [37]:
training.head()

,amount_tsh,days_since_recorded,funder,gps_height,installer,basin,subvillage,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type,payment_type,water_quality,quantity_group,source_type,source_class,waterpoint_type,waterpoint_type_group
0,62,995,8,1390,7,1,11807,109,1,6,2246,0,5,1,0,6,1,6,0,1,1
1,0,272,8,1399,7,4,15838,280,2,2,2171,1,1,1,2,6,2,3,1,1,1
2,13,281,8,686,7,5,9074,250,1,6,2121,1,0,1,5,6,1,1,1,2,1
3,0,309,3,263,7,7,8982,58,1,6,2171,1,4,8,2,6,0,0,0,2,1
4,0,874,8,0,7,4,7698,0,1,2,2171,1,6,1,2,6,3,3,1,1,1


In [18]:
test.head()

,amount_tsh,days_since_recorded,funder,gps_height,installer,basin,subvillage,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type,payment_type,water_quality,quantity_group,source_type,source_class,waterpoint_type,waterpoint_type_group
0,0,302,5,1996,3,6,5937,321,1,8,0,1,2,3,4,2,0,6,2,0,0
1,0,302,0,1569,5,0,142,300,1,5,1668,1,0,0,4,2,3,3,0,1,1
2,0,305,5,1567,3,6,6488,500,1,5,496,0,2,3,4,2,3,6,2,0,0
3,0,315,5,267,3,8,974,250,0,5,0,1,4,3,3,2,4,5,0,0,0
4,65,251,5,1260,3,8,629,60,0,9,225,1,0,0,6,2,2,3,0,1,1


In [19]:
## Converting the Training dataframe into a matrix and predictor as y 
X = training.as_matrix()
y = train["status_group"].tolist()
X_pred=test.as_matrix()

In [20]:
X

array([[  77,  995,    5, ...,    0,    1,    1],
       [   0,  272,    5, ...,    2,    1,    1],
       [  15,  281,    5, ...,    2,    3,    1],
       ..., 
       [   0,  967,    5, ...,    0,    4,    3],
       [   0, 1001,    5, ...,    0,    4,    3],
       [   0,  986,    4, ...,    0,    4,    3]], dtype=int64)

In [21]:
import sklearn.model_selection 
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, 
                                                                             y, 
                                                                             test_size = 0.3, 
                                                                             random_state = 0)

In [22]:
import sklearn.ensemble

rfc = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, 
                                              min_samples_split=6,
                                              criterion='gini', 
                                              max_features='auto',
                                              oob_score=True,
                                              random_state=1,
                                              n_jobs=-1)

In [23]:
rfc.fit(X_train, y_train)

print('Random Forest Classifier Train Accuracy Score :', rfc.score(X_train, y_train))
print('Random Forest Classifier Test Score :', rfc.score(X_test, y_test))

Random Forest Classifier Train Accuracy Score : 0.939898989899
Random Forest Classifier Test Score : 0.804994388328


In [24]:
importances=list(zip(rfc.feature_importances_,training.columns))

In [25]:
importances.sort(reverse=True)

In [26]:
importances

[(0.15725514639732438, 'quantity_group'),
 (0.10464788138305624, 'days_since_recorded'),
 (0.097236886920257962, 'subvillage'),
 (0.077578339634125804, 'gps_height'),
 (0.057807339765938055, 'waterpoint_type'),
 (0.053667491168830496, 'population'),
 (0.048538554692598913, 'scheme_name'),
 (0.046945906942469411, 'waterpoint_type_group'),
 (0.040319363045057573, 'extraction_type'),
 (0.03702482126805616, 'construction_year'),
 (0.036400446481427116, 'basin'),
 (0.036281669364962743, 'payment_type'),
 (0.03482203637226125, 'amount_tsh'),
 (0.031269029504350357, 'funder'),
 (0.030401456471061471, 'installer'),
 (0.030012882125642085, 'scheme_management'),
 (0.025658030776569102, 'source_type'),
 (0.016331109837856902, 'water_quality'),
 (0.01535688594203522, 'permit'),
 (0.013598752361134345, 'public_meeting'),
 (0.0088459695449845301, 'source_class')]

# First y_pred

In [27]:
y_pred=rfc.predict(X_pred)

In [29]:
pd.DataFrame(y_pred).to_csv('Y_pred.csv',index=True)

# K-fold CV

In [34]:
from sklearn.model_selection import cross_val_score


In [35]:
accuracies=cross_val_score(estimator=rfc,X=X_train, y =y_train, cv=10 ,verbose=1 )


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  5.1min finished


In [36]:
accuracies

array([ 0.8050012 ,  0.80788651,  0.80187545,  0.80860784,  0.8023088 ,
        0.7977393 ,  0.8044733 ,  0.79841232,  0.80394515,  0.79932628])

In [50]:
#Using StandardScaler
from sklearn.preprocessing import StandardScaler

In [53]:
ss=StandardScaler()

In [54]:
X=ss.fit_transform(training)

In [55]:
X

array([[ 3.35282034,  1.14113571,  0.50520616, ..., -0.53841035,
        -0.8836145 , -0.76596738],
       [-0.55432861, -1.02215173,  0.50520616, ...,  1.76507387,
        -0.8836145 , -0.76596738],
       [ 0.2649123 , -0.99522284,  0.50520616, ...,  1.76507387,
        -0.32226541, -0.76596738],
       ..., 
       [-0.55432861,  1.05735695,  0.50520616, ..., -0.53841035,
         0.80043277,  0.68443707],
       [-0.55432861,  1.15908831,  0.50520616, ..., -0.53841035,
         0.80043277,  0.68443707],
       [-0.55432861,  1.11420682,  1.55956798, ..., -0.53841035,
         0.80043277,  0.68443707]])

In [56]:
import sklearn.model_selection 
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, 
                                                                             y, 
                                                                             test_size = 0.3, 
                                                                             random_state = 0)

In [57]:
import sklearn.ensemble

rfc = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, 
                                              min_samples_split=6,
                                              criterion='gini', 
                                              max_features='auto',
                                              oob_score=True,
                                              random_state=1,
                                              n_jobs=-1)

In [58]:
rfc.fit(X_train, y_train)

print('Random Forest Classifier Train Accuracy Score :', rfc.score(X_train, y_train))
print('Random Forest Classifier Test Score :', rfc.score(X_test, y_test))

Random Forest Classifier Train Accuracy Score : 0.938552188552
Random Forest Classifier Test Score : 0.805050505051


# Using PCA

In [66]:
from sklearn.decomposition import PCA

In [67]:
from sklearn.pipeline import Pipeline

In [69]:
pca=PCA()
pipe=Pipeline([('pca',pca),('rfc',rfc)])

In [70]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('rfc', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
         ...timators=1000, n_jobs=-1, oob_score=True, random_state=1,
            verbose=0, warm_start=False))])

In [71]:
pipe.score(X_test,y_test)

0.77676767676767677